In [3]:
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor,ExtraTreesClassifier,RandomForestClassifier
from sklearn.preprocessing import StandardScaler,KBinsDiscretizer,LabelEncoder,MinMaxScaler
from sklearn.metrics import mean_squared_error,f1_score,confusion_matrix,log_loss
from sklearn.kernel_approximation import Nystroem
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA,NMF

from catboost import Pool, cv,CatBoostClassifier,CatBoostRegressor
from catboost.eval.catboost_evaluation import *



from tqdm import tqdm


In [4]:
train_df = pd.read_csv('train_df_final.csv')
train_df = train_df.fillna(0)
train_df_org = pd.read_csv('train_df_final_blanced.csv')
train_df_org = train_df_org.fillna(0)
test_df = pd.read_csv('test_df_final.csv')
test_df = test_df.fillna(0)
submission_df = pd.read_csv('sample_submission.csv')

In [5]:
learn_params = {'iterations': 2000, 
                'random_seed': 0, 
                'logging_level': 'Silent',
                'loss_function': 'Logloss',
                # You could set learning process to GPU
                # 'devices': '1',  
                'task_type': 'GPU',
                'loss_function' : 'Logloss',
                'boosting_type': 'Plain', 
                # For feature evaluation learning time is important and we need just the relative quality
                'max_ctr_complexity' : 4}

In [6]:
help(CatboostEvaluation)

Help on class CatboostEvaluation in module catboost.eval.catboost_evaluation:

class CatboostEvaluation(builtins.object)
 |  CatboostEvaluation(path_to_dataset, fold_size, fold_count, column_description, fold_offset=0, group_column=None, working_dir=None, remove_models=True, delimiter='\t', has_header=False, partition_random_seed=0, min_fold_count=1)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, path_to_dataset, fold_size, fold_count, column_description, fold_offset=0, group_column=None, working_dir=None, remove_models=True, delimiter='\t', has_header=False, partition_random_seed=0, min_fold_count=1)
 |      Args:
 |          :param path_to_dataset: (str) Path to the dataset to be used for evaluation.
 |          :param fold_size: (int) Size of the folds in cross-validation.
 |          :param fold_count: (int) Number of times we get a new fold, learn a model and check results as if
 |          there wouldn't be any offset. If there'are some offset it means that the real count